In [5]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyelty and friendliness.",
        metadata={"source":"mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pet that often enjoy their own space",
        metadata={"source":"mammals-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners requiring relative simple care",
        metadata={"source":"fish-pets-doc"}
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimiking human speech",
        metadata={"source":"birds-pets-doc"}
    ),
    Document(
        page_content="Rabbits are social animal that needs plenty of space",
        metadata={"source":"mammals-pets-doc"}
    )
]




In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

if not groq_api_key:
    raise ValueError("GROQ_API_KEY not found. Check your .env file.")

print("Groq key loaded successfully")

Groq key loaded successfully


In [3]:
from langchain_chroma import Chroma
from langchain_groq import ChatGroq

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")

llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x1244d0520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1244d0430>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

vector_store = Chroma.from_documents(documents=documents, embedding=embeddings)

vector_store

In [7]:
vector_store.similarity_search("cat")

[Document(id='bfc206ab-c9ef-4e23-be9a-19a0804ecd6b', metadata={'source': 'mammals-pets-doc'}, page_content='Cats are independent pet that often enjoy their own space'),
 Document(id='3601282e-88d4-41a2-974c-035e717dd047', metadata={'source': 'mammals-pets-doc'}, page_content='Rabbits are social animal that needs plenty of space'),
 Document(id='995177e4-9454-4281-bc26-118db3a716b0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyelty and friendliness.'),
 Document(id='e1883450-1d5c-4d57-8e98-46232f645c53', metadata={'source': 'birds-pets-doc'}, page_content='Parrots are intelligent birds capable of mimiking human speech')]

In [8]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(
    vector_store.similarity_search
).bind(k=1)

retriever.batch(["cat","dog"])

[[Document(id='bfc206ab-c9ef-4e23-be9a-19a0804ecd6b', metadata={'source': 'mammals-pets-doc'}, page_content='Cats are independent pet that often enjoy their own space')],
 [Document(id='995177e4-9454-4281-bc26-118db3a716b0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyelty and friendliness.')]]

In [9]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2}
)

retriever.batch(["cat","dog"])

[[Document(id='bfc206ab-c9ef-4e23-be9a-19a0804ecd6b', metadata={'source': 'mammals-pets-doc'}, page_content='Cats are independent pet that often enjoy their own space'),
  Document(id='3601282e-88d4-41a2-974c-035e717dd047', metadata={'source': 'mammals-pets-doc'}, page_content='Rabbits are social animal that needs plenty of space')],
 [Document(id='995177e4-9454-4281-bc26-118db3a716b0', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyelty and friendliness.'),
  Document(id='bfc206ab-c9ef-4e23-be9a-19a0804ecd6b', metadata={'source': 'mammals-pets-doc'}, page_content='Cats are independent pet that often enjoy their own space')]]

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """ 
            Answer this question using the provided context only.

            {question}

            Context:

            {context}
"""

prompt = ChatPromptTemplate.from_messages(
    [("human",message)]
)

rag_chain = {
    "context": retriever,
    "question": RunnablePassthrough()
}|prompt|llm

In [11]:
response = rag_chain.invoke("Tell Me About Dogs!")

response.content

'Dogs are great companions, known for their loyalty and friendliness.'